# Case 5 

> In this case study, we are exploring deep learning interpretability on BSI prognosis, before, during and after training a CNN model on multi-variate time series medical record.
 
> we use hourly aggregated physiological data and lab results from 10,000 ICU patients. 1824 positive events and 31496 control group events are generated from the population. 

> we use 25 features selected by domain knowledge, 96 hours prior to the blood culture acquisition to classify positive test results from the control group -- negative test results and 4 days of randomly sliced baseline patient data.  


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import FAIRStream

In [2]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'
# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_fwd'

# initiate dataframe to return
all_df=None
# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
print(bsi_stream.engineer.variable_dict)
print(bsi_stream.engineer.csv_source_dict)

Success: Querier has initiated a csv source dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_fwd/meta_data/csv_source_dict_demo.json
Success: Querier has initiated a variable dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_fwd/meta_data/variable_dict_demo.json
Unable to read sql source dictionary. Use Querier.update_sql_source_dict() to build one.
{'__uid': {'src_names': ['id', 'ID', 'subject_id', 'subject_id'], 'label': 'subject id', 'unique_per_sbj': True}, '__time': {'src_names': ['tsa', 'timeMinutes'], 'label': 'Time since admission', 'unit': 'minute'}, '__anchor': {'src_names': ['True_positive', 'True positive'], 'label': 'episode anchor', 'unique_per_sbj': False, 'factor': {'levels': {'__nbc': ['nan'], '__neg': ['0', '0.0'], '__pos': ['1', '1.0']}, 'impute_per_sbj': {'nan_level': '__nbc'}}}, 'y': {'output': True, 'src_names': ['True_positive', 'True positive'], 'label': 'Blood culture result', 'unique_per_sbj': False, '

In [ ]:
# bsi_stream.querier.create_csv_pool(csv_pool_dir = csv_pool_path,
#                                   source_key="uvanewbc",
#                                   file_key="all",
#                                   sep="_")

In [ ]:
# bsi_stream.querier.create_csv_pool(csv_pool_dir = csv_pool_path,
#                                   source_key="uvanewnbc",
#                                   file_key="all",
#                                   sep="_")

## nbc (baseline) group 

In [3]:
#all_df = pd.DataFrame()
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=1*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=3*24*60, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=4*24*60) # the minimum distance between two episodes

Success! Engineer has updated attributes --- episode. 


In [4]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 5000, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median',
                              sep="_", 
                              return_episode=True)# imputation on response (no need in BSI project)


Engineer is sampling without replacement --- 
5000---out of---14971---subjects are sampled from csv pool of size---51911
-- __time fixed
-- __anchor fixed
-- y fixed
--- fix upper boundary for age by 90.0
--- fix lower boundary for age by 18.05
-- age fixed
--- fix upper boundary for albumin by 5.0
--- fix lower boundary for albumin by 1.2
-- albumin fixed
--- fix upper boundary for alp by 400.9482
--- fix lower boundary for alp by 25.286364000000003
-- alp fixed
--- fix upper boundary for alt by 1085.0
--- fix lower boundary for alt by 6.0
-- alt fixed
--- fix upper boundary for ast by 1300.0
--- fix lower boundary for ast by 9.0
-- ast fixed
--- fix upper boundary for bicarbonate by 53.20671999999832
--- fix lower boundary for bicarbonate by 5.3
-- bicarbonate fixed
--- fix upper boundary for bun by 150.0
--- fix lower boundary for bun by 2.0
-- bun fixed
--- fix upper boundary for calcium by 13.6
--- fix lower boundary for calcium by 5.2
-- calcium fixed
--- fix upper boundary for c

--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_7260
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_25997
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_7727
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_30760
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_18566
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_13734
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_6747
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_22255
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_2817
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_4208
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc

--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_1681
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_11749
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_26584
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_14875
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_31285
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_15031
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_2474
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_9785
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_31034
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_14956
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewn

--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_9839
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_25796
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_3633
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_31517
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_32750
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_15791
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_31155
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_3254
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_30143
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_26514
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewn

--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_15369
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_28617
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_30042
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_15493
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_10191
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_15934
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_28656
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_19011
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_6963
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvanewnbc_31955
--- fix out-of-dictionry level/orders --- ['nan']--- with NA for subject ---uvane

...

Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_22501
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_12748
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_29613
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_10392
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_21742
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes 

Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_19982
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_17147
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_1091
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_31536
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_32205
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes f

Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_20066
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_450
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_28959
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_5174
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_29698
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for

Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_26697
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_3145
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_29209
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_8000
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_14050
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes fo

Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_29252
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_28692
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_1922
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_17366
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes for uvanewnbc_27653
Success! Output/responce variable mean in current sample space  --- 
y___pos    0.0
y___neg    0.0
y___nbc    1.0
dtype: float64
--- prepare episodes f

In [5]:
all_df_new = pd.concat( [bsi_stream.engineer.train_df, bsi_stream.engineer.valid_df], axis=0)
all_df_new = pd.concat( [all_df_new, bsi_stream.engineer.test_df], axis=0)
# filter 3 days after the anchor [0, 72]
all_df_new = all_df_new.loc[all_df_new.__ep_relative_time>=0,:]
# shift ep relative time by 48 hours [-48, 24]
all_df_new.__ep_relative_time = all_df_new.__ep_relative_time - 2*24*60
all_df_new.describe()

,age,albumin,alp,alt,ast,bicarbonate,bun,calcium,chloride,co2,creatinine,dbp,fio2_pct,glucose,hematocrit,hemoglobin,heart_rate,lactic_acid,magnesium,o2_flow,pco2,peep,ph_arterial,phosphorus,po2,potassium,protime_inr,ptt,resp_rate,sbp,sodium,spo2,temp,total_bilirubin,total_protein,txp___yes,txp___no,troponin,wbc,gram_positive___yes,gram_positive___no,gram_negative___no,gram_negative___yes,fungus___no,fungus___yes,mycobacterial___no,mycobacterial___yes,y___pos,y___neg,y___nbc,__time_bin,__ep_relative_time,__ep_order
count,601632.000000,181936.000000,881.000000,167890.000000,177131.000000,104816.000000,411185.000000,411362.000000,411287.000000,411142.000000,411117.000000,556810.000000,210048.000000,416629.000000,403428.000000,404109.000000,579923.000000,74551.000000,368159.000000,324490.000000,104867.000000,136022.000000,516.000000,324230.000000,109127.000000,411930.000000,175770.000000,138563.000000,172826.000000,556810.000000,416398.000000,578928.000000,462681.000000,181412.000000,180889.000000,601632.000000,601632.000000,47796.000000,395845.000000,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.0,601632.000000,601632.000000,601632.000000
mean,62.904637,3.053817,95.598206,84.471527,113.677885,23.445684,23.821974,8.528157,104.306851,24.024620,1.291624,66.308395,41.666692,135.437015,31.830345,10.496875,82.167866,2.042709,1.997766,5.087936,40.906672,6.620397,7.370313,3.393205,101.011487,4.063380,1.325369,32.135789,19.259738,124.375647,137.471747,96.083055,98.299449,1.497406,5.594483,0.046553,0.953447,4.899771,11.306352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3021.672451,-750.000000,1.699737
std,16.359145,0.497742,89.515312,188.832159,214.945507,4.672838,18.503428,0.688907,5.858657,4.422262,1.215664,12.046026,19.388142,47.548159,6.657095,2.262707,17.225658,2.096487,0.363746,8.852637,9.366923,2.458203,0.072178,1.104053,59.024963,0.518798,0.509742,15.995912,5.888993,22.840439,4.598896,3.124682,0.864433,3.559488,0.889046,0.210680,0.210680,9.124423,5.564610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8693.822743,1246.957331,1.221597
min,18.050000,1.200000,25.286364,6.000000,9.000000,5.300000,2.000000,5.200000,73.000000,7.000000,0.200000,26.000000,21.000000,40.763500,14.400000,4.700000,30.000000,0.300000,0.900000,0.000000,16.300000,0.000000,7.147114,1.000000,18.800000,2.200000,0.800000,20.100000,6.000000,40.000000,112.000000,79.000000,82.901670,0.100000,2.400000,0.000000,0.000000,0.020000,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-130.000000,-2880.000000,1.000000
25%,53.979167,2.700000,54.000000,14.000000,30.000000,21.000000,12.000000,8.100000,101.000000,21.000000,0.700000,58.000000,30.000000,105.000000,26.400000,8.700000,70.000000,1.000000,1.800000,2.000000,35.600000,5.000000,7.332000,2.700000,71.400000,3.700000,1.100000,25.600000,15.500000,108.000000,135.000000,95.000000,97.900000,0.500000,5.000000,0.000000,1.000000,0.060000,7.675000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,25.000000,-1815.000000,1.000000
50%,65.270000,3.100000,66.000000,24.000000,48.000000,23.200000,18.000000,8.500000,105.000000,24.000000,0.900000,64.000000,40.000000,125.000000,31.000000,10.200000,80.312500,1.500000,2.000000,2.000000,39.700000,5.000000,7.391000,3.200000,88.900000,4.000000,1.200000,28.000000,18.000000,122.000000,138.000000,96.500000,98.200000,0.700000,5.500000,0.000000,1.000000,0.290000,10.310000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,52.000000,-750.000000,1.000000
75%,75.130000,3.400000,93.000000,54.000000,87.000000,25.500000,29.000000,8.900000,108.000000,26.000000,1.300000,73.000000,42.000000,152.000000,36.600000,12.100000,93.000000,2.200000,2.200000,4.000000,44.300000,8.000000,7.416000,3.900000,114.000000,4.400000,1.300000,31.600000,22.000000,139.000000,140.000000,98.000000,98.700000,1.200000,6.200000,0.000000,1.000000,4.080000,13.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,150.000000,315.000000,2.000000
max,90.000000,5.000000,400.948200,1085.000000,1300.000000,53.206720

In [6]:
all_df = pd.read_csv("./whole_df_fwd_uvanew_nbc.csv")
print(len(set(all_df.__uid))) 
all_df_new = all_df_new[~all_df_new["__uid"].isin(list(set(all_df["__uid"])))]  
all_df_new.describe()

10732


,age,albumin,alp,alt,ast,bicarbonate,bun,calcium,chloride,co2,creatinine,dbp,fio2_pct,glucose,hematocrit,hemoglobin,heart_rate,lactic_acid,magnesium,o2_flow,pco2,peep,ph_arterial,phosphorus,po2,potassium,protime_inr,ptt,resp_rate,sbp,sodium,spo2,temp,total_bilirubin,total_protein,txp___yes,txp___no,troponin,wbc,gram_positive___yes,gram_positive___no,gram_negative___no,gram_negative___yes,fungus___no,fungus___yes,mycobacterial___no,mycobacterial___yes,y___pos,y___neg,y___nbc,__time_bin,__ep_relative_time,__ep_order
count,171504.000000,51885.000000,215.000000,48355.000000,50410.000000,29401.000000,116214.000000,116272.000000,116238.000000,116188.000000,116148.000000,159057.000000,61030.000000,117634.000000,114248.000000,114505.000000,165169.000000,22909.000000,104121.000000,94739.000000,29401.000000,38988.000000,96.000000,91240.000000,31168.000000,116345.000000,50257.000000,40218.000000,49470.000000,159057.000000,117629.000000,164830.000000,131288.000000,51700.000000,51632.000000,171504.000000,171504.000000,13621.000000,112332.000000,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.0,171504.000000,171504.00000,171504.000000
mean,62.535762,3.064565,47.428647,84.014683,117.756526,23.201517,23.486684,8.519766,104.272643,23.992207,1.273262,66.457779,42.017559,134.904910,31.966152,10.534258,82.240208,1.937339,2.006203,5.388825,40.741106,6.716414,7.355381,3.394738,101.810568,4.065343,1.355866,32.418074,19.178764,124.949741,137.448281,96.091519,98.300438,1.631435,5.630377,0.051217,0.948783,4.763252,11.387332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3437.121746,-750.00000,1.718304
std,16.440013,0.496113,16.423221,188.655694,225.529222,4.466630,18.178273,0.684040,5.816941,4.538418,1.178437,11.830084,19.553911,46.610179,6.708096,2.273768,17.294667,1.854826,0.360170,9.184377,8.920334,2.495248,0.057778,1.088056,63.279710,0.516137,0.591794,16.065217,5.805359,22.675211,4.457327,3.105237,0.864775,3.939503,0.903482,0.220442,0.220442,9.329386,5.392860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9232.849085,1246.95993,1.217698
min,18.290000,1.400000,25.286364,6.000000,9.000000,5.300000,2.000000,5.200000,73.000000,7.000000,0.200000,26.000000,21.000000,40.763500,14.400000,4.700000,30.000000,0.300000,0.900000,0.000000,16.300000,0.000000,7.147114,1.000000,18.800000,2.200000,0.800000,20.100000,6.000000,40.000000,112.000000,79.000000,82.901670,0.100000,2.400000,0.000000,0.000000,0.020000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-130.000000,-2880.00000,1.000000
25%,53.500000,2.800000,35.000000,14.000000,30.000000,20.900000,12.000000,8.100000,101.000000,21.000000,0.700000,58.000000,30.000000,105.000000,26.700000,8.700000,70.500000,1.000000,1.800000,2.000000,35.800000,5.000000,7.305000,2.700000,71.100000,3.700000,1.100000,25.600000,15.666667,108.000000,135.000000,95.000000,97.900000,0.500000,5.000000,0.000000,1.000000,0.050000,7.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.000000,-1815.00000,1.000000
50%,64.760000,3.100000,37.000000,23.000000,46.000000,23.100000,18.000000,8.500000,105.000000,24.000000,0.900000,65.000000,40.000000,124.000000,31.000000,10.200000,80.500000,1.400000,2.000000,2.000000,39.700000,5.000000,7.396000,3.300000,89.600000,4.000000,1.200000,28.100000,18.000000,123.000000,138.000000,96.500000,98.200000,0.700000,5.500000,0.000000,1.000000,0.220000,10.470000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.000000,-750.00000,1.000000
75%,74.980000,3.400000,63.000000,49.000000,87.375000,25.300000,29.000000,9.000000,108.000000,26.000000,1.300000,73.000000,44.375000,151.000000,36.800000,12.200000,93.000000,2.200000,2.200000,4.000000,44.400000,8.000000,7.414000,3.900000,114.100000,4.400000,1.300000,31.600000,22.000000,139.000000,140.000000,98.000000,98.700000,1.100000,6.200000,0.000000,1.000000,2.130000,13.850000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,164.000000,315.00000,2.000000
max,90.000000,5.000000,66.000000,1085.000000,1300.000000,53.206720,150.000000,13.400000,13

In [7]:
if all_df is None:
    all_df = all_df_new
else:
    all_df = pd.concat( [all_df, all_df_new], axis=0)
print(len(set(all_df.__uid))) 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


12141


In [ ]:
all_df.to_csv("./whole_df_fwd_uvanew_nbc.csv",index=False)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import FAIRStream

In [ ]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'
# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_txp'

In [ ]:
# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
bsi_stream.engineer.variable_dict

In [ ]:
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=2*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=24*60, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=7*24*60) # the minimum distance between two episodes

In [ ]:
print(bsi_stream.engineer.episode)

In [ ]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 6000, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median',
                              sep="_")# imputation on response (no need in BSI project)
# please see the end of console 
# --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 

In [ ]:
all_df = pd.concat( [bsi_stream.engineer.train_df, bsi_stream.engineer.valid_df], axis=0)
all_df = pd.concat( [all_df, bsi_stream.engineer.test_df], axis=0)
print(len(set(all_df.__uid)))
all_df.describe()

In [ ]:
all_df.to_csv("./whole_df_uvanew.csv", index=False)

In [ ]:
from random import randint
import matplotlib.pylab as plt 
import numpy as np
from tensorflow.keras.models import Sequential, Model

def get_feature_maps(model, layer_id, input_image):
    model_ = Model(inputs=[model.input], outputs=[model.layers[layer_id].output])
    return model_.predict(np.expand_dims(input_image, axis=0))[0,:,:,:].transpose((2,1,0))




def plot_features_map(input_image_list, 
                      cnn,
                      img_title_list,
                      layer_idx):
    
    nrow = len(layer_idx)
    ncol = len(input_image_list)
    
    fig, ax = plt.subplots(nrow+1, ncol, figsize=(20,20))
    
    for j in range(ncol):
        input_image = input_image_list[j]
        ax[0][j].imshow(input_image[:,:,0].transpose(1,0))
        ax[0][j].set_title(img_title_list[j])
        for i in range(nrow):
            feature_map = get_feature_maps(cnn, layer_idx[i], input_image)
            ax[i+1][j].imshow(feature_map[0,:,:])
            ax[i+1][j].set_title('layer {} - {}'.format(layer_idx[i], cnn.layers[layer_idx[i]].get_config()['name']))

    return

## setup

In [ ]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'


# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case4'

In [ ]:
# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
bsi_stream.engineer.variable_dict

## Define Episode

In [ ]:
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=4*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=1, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=7*24*60) # the minimum distance between two episodes

## Build MVTS (multi-variable time series) data objects
- train_df_imputed, valid_df_imputed and test_df_imputed are dataframes
- train_tfds, valid_tfds and test_tfds are tensorflow datasets

In [ ]:
# # Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
# bsi_stream.engineer.BuildMVTS(csv_pool_path, 
#                               nsbj = 10, # number of subjects / patients to sample from the pool 
#                               valid_frac = 0.2, # fraction of number of subjects in validation dataset
#                               test_frac = 0.1, # fraction of number of subjects in left-out test dataset
#                               batch_size = 64, # batch size (usually 32,64,128..)
#                               impute_input='median', # imputation on predictors
#                               impute_output='median' )# imputation on response (no need in BSI project)

# # please see the end of console 
# # --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# # --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 

### print all the attributes of the engineer

In [ ]:
# bsi_stream.engineer.info()

# # extract X, Y numpy array by engineer build-in function
# X_train, Y_train, X_valid, Y_valid, X_test, Y_test = bsi_stream.engineer.ExtractXY()

# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
# X_valid = X_valid.reshape(X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
# X_train = X_train.astype('float32')
# X_valid = X_valid.astype('float32')
# X_test = X_test.astype('float32')

# Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[2])
# Y_valid = Y_valid.reshape(Y_valid.shape[0], Y_valid.shape[2])
# Y_test = Y_test.reshape(Y_test.shape[0], Y_test.shape[2])

# X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
# Y_all = np.concatenate((Y_train, Y_valid, Y_test), axis=0)


In [ ]:
# specify input column names
bsi_stream.engineer.input_vars = ['age___vital', 'age___lab', 'temp___vital', 'heart_rate___vital', 'systolic_blood_pressure___vital', 'diastolic_blood_pressure___vital', 'resp_rate___vital', 'glucose___vital', 'bilirubin___vital', 'potassium___vital', 'albumin___vital', 'calcium___vital', 'wbc___vital', 'creatinine___vital', 'platelet_count___vital', 'alt___vital', 'alp___vital', 'ast___vital', 'paco2___vital', 'chloride___vital', 'troponin___vital', 'ptt___vital', 'lactate___vital', 'bun___vital', 'magnesium___vital']


## --- Pre-modeling Interpretability (raw scale) ---

In [ ]:
X_all = np.load("./X_all.npy")
Y_all = np.load("./Y_all.npy")
X_train = np.load("./X_train.npy")
Y_train = np.load("./Y_train.npy")
X_valid = np.load("./X_valid.npy")
Y_valid = np.load("./Y_valid.npy")
X_test = np.load("./X_test.npy")
Y_test = np.load("./Y_test.npy")

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)
print("X_all shape", X_all.shape)
print("Y_all shape", Y_all.shape)

X_pos = X_all[Y_all[:,1]==1.0][:,:,:,0]
X_ctrl = X_all[Y_all[:,1]==0.0][:,:,:,0]
print("X_pos shape", X_pos.shape)
print("X_ctrl shape", X_ctrl.shape)


### 1. print a few episode as images

In [ ]:
fig, ax = plt.subplots(1,10,figsize=(20,20))
axes = ax.flatten()
for i in range(10): 
    norm = colors.TwoSlopeNorm(vmin = -np.max(np.abs(X_pos[i])), 
                               vcenter = 0,  # change zero to be not white 
                               vmax = np.max(np.abs(X_pos[i])) )
    axes[i].imshow(X_pos[i], cmap='bwr', norm=norm)
    axes[i].set_title('postive episode {}'.format(i))

fig, ax = plt.subplots(1,10,figsize=(20,20))
axes = ax.flatten()
for i in range(10):
    norm = colors.TwoSlopeNorm(vmin = -np.max(np.abs(X_pos[i])), 
                               vcenter = 0, 
                               vmax = np.max(np.abs(X_pos[i])) )
    axes[i].imshow(X_ctrl[i], cmap='bwr', norm=norm)
    axes[i].set_title('control episode {}'.format(i))

### 2. Visualize Statistical Moments -- Positive VS Control Group

#### Statistic Moment 1 -- mean

In [ ]:
# outlier in image

In [ ]:
X_pos_stat = X_pos.mean(axis=0).transpose((1,0))
X_ctrl_stat = X_ctrl.mean(axis=0).transpose((1,0))
X_all_stat = X_all.mean(axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()

im = ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[0])

im = ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[1])

im = ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[2])


#### Statistic Moment 2 -- std

In [ ]:
X_pos_stat = X_pos.std(axis=0).transpose((1,0))
X_ctrl_stat = X_ctrl.std(axis=0).transpose((1,0))
X_all_stat = X_all.std(axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()
im=ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[0])

im=ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[1])

im=ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[2])


#### Statistic Moment 3 -- skewness

In [ ]:
X_pos_stat = scipy.stats.skew(X_pos, axis=0).transpose((1,0))
X_ctrl_stat = scipy.stats.skew(X_ctrl, axis=0).transpose((1,0))
X_all_stat = scipy.stats.skew(X_all, axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()
im=ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[0])

im=ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[1])

im=ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[2])

#### Statistic Moment 4 -- Kurtosis

In [ ]:
X_pos_stat = scipy.stats.kurtosis(X_pos, axis=0).transpose((1,0))
X_ctrl_stat = scipy.stats.kurtosis(X_ctrl, axis=0).transpose((1,0))
X_all_stat = scipy.stats.kurtosis(X_all, axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()
im=ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[0])

im=ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[1])

im=ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[2])

## --- Modeling interpretation raw scale (CNN) ---

In [ ]:
X_train_2d = X_train[:,:,:,0].reshape(X_train[:,:,:,0].shape[0],-1)
X_valid_2d = X_valid[:,:,:,0].reshape(X_valid[:,:,:,0].shape[0],-1)
Y_train_binary = Y_train[:,1]
Y_valid_binary = Y_valid[:,1]
print(X_train_2d.shape)
print(Y_train_binary.shape)
print(X_valid_2d.shape)
print(Y_valid_binary.shape)

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='liblinear').fit(X_train_2d, Y_train_binary)

#to check the shape of the coefficient matrix
clf.coef_.shape

In [ ]:
scale = np.max(np.abs(clf.coef_))

p = plt.figure(figsize=(25, 25));
nclasses=1
for i in range(nclasses):
    p = plt.subplot(1, nclasses, i + 1)
    p = plt.imshow(clf.coef_[i].reshape(96, 25).T,
                  cmap=plt.cm.RdBu, vmin=-scale, vmax=scale);
    p = plt.axis('off')
    p = plt.title('positive');

### AUROC for logistic regression

In [ ]:
Y_valid_pred = clf.predict_proba(X_valid_2d)
sklearn.metrics.roc_auc_score(Y_valid_binary, Y_valid_pred[:,1])

### binary setup logistic regression

In [ ]:
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC'),
    keras.metrics.AUC(name='AUPRC', curve='PR')
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.Flatten(input_shape=list(X_train.shape)[1:4]),
    keras.layers.Dense(1, activation='sigmoid')
])
mdl.summary()
mdl.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=5)
his = mdl.fit(X_train, Y_train_binary, 
              epochs=50, 
              validation_data=(X_valid,Y_valid_binary), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(X_train, Y_train_binary, 
              epochs=50, 
              validation_data=(X_valid,Y_valid_binary), callbacks=[callback])

### multiple classification setup logistic regression

In [ ]:
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC', multi_label=True),
    keras.metrics.AUC(name='AUPRC', curve='PR', multi_label=True),
    #keras.metrics.recall(....),
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.Flatten(input_shape=list(X_train.shape)[1:4]),
    keras.layers.Dense(2, activation='softmax')
])
mdl.summary()
mdl.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=5)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])

In [ ]:
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC', multi_label=True),
    keras.metrics.AUC(name='AUPRC', curve='PR', multi_label=True),
    keras.metrics.recall(....),
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.Conv2D(64, 
                        kernel_size=(3,3),
                        activation='relu', 
                        padding='same',
                        input_shape=list(X_train.shape)[1:4]),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, 
                        kernel_size=(3,3),
                        activation='relu', 
                        padding='same'),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(1, 
                        kernel_size=(3,3),
                        activation='relu', 
                        padding='same'),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
mdl.summary()
mdl.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=5)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])

In [ ]:
X_train_pos_img = X_train[Y_train[:,0]==1.0].mean(axis=0)
X_train_ctrl_img = X_train[Y_train[:,0]==0.0].mean(axis=0)

X_valid_pos_img = X_valid[Y_valid[:,0]==1.0].mean(axis=0)
X_valid_ctrl_img = X_valid[Y_valid[:,0]==0.0].mean(axis=0)

X_all_pos_img = X_all[Y_all[:,0]==1.0].mean(axis=0)
X_all_ctrl_img = X_all[Y_all[:,0]==0.0].mean(axis=0)

plot_features_map(input_image_list=[X_all_pos_img, X_all_ctrl_img,
                                    X_train_pos_img, X_train_ctrl_img, 
                                    X_valid_pos_img, X_valid_ctrl_img],
                  img_title_list=["POS (all)", "CTRL (all)",
                                  "POS (train)", "CTRL (train)", 
                                  "POS (valid)", " CTRL (valid)"],
                  layer_idx=[0,3,6], 
                  cnn=mdl)

## --- Post-Modeling interpretation (raw scale + saliency map) ---

In [ ]:
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations
def plot_saliency(img_idx):
    grads = visualize_saliency(mdl,
                               "visualized_layer",
                               filter_indices=Y_all[img_idx][0], 
                               seed_input=X_all[img_idx], 
                               backprop_modifier=None,
                               grad_modifier="absolute")
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    ax[0].imshow(X_all[img_idx])
    ax[0].set_title('original img id {}'.format(img_idx))
    ax[1].imshow(grads, cmap='jet')
    ax[1].set_title('saliency - predicted {}'.format(img_idx))

plot_saliency(img_idx = 0)

## --- Pre-training Interpretability (standardized) ---

In [ ]:
X_all = np.load("./X_all.npy")
Y_all = np.load("./Y_all.npy")
X_train = np.load("./X_train.npy")
Y_train = np.load("./Y_train.npy")
X_valid = np.load("./X_valid.npy")
Y_valid = np.load("./Y_valid.npy")
X_test = np.load("./X_test.npy")
Y_test = np.load("./Y_test.npy")

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)
print("X_all shape", X_all.shape)
print("Y_all shape", Y_all.shape)

X_train = (X_train - X_all.mean(axis=0))/X_all.std(axis=0)
X_valid = (X_valid - X_all.mean(axis=0))/X_all.std(axis=0)
X_all = (X_all - X_all.mean(axis=0))/X_all.std(axis=0)

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_all shape", X_all.shape)
print("Y_all shape", Y_all.shape)


X_pos = X_all[Y_all[:,1]==1.0][:,:,:,0]
X_ctrl = X_all[Y_all[:,1]==0.0][:,:,:,0]
print("X_pos shape", X_pos.shape)
print("X_ctrl shape", X_ctrl.shape)


In [ ]:
fig, ax = plt.subplots(1,10,figsize=(20,20))
axes = ax.flatten()
for i in range(10): 
    norm = colors.TwoSlopeNorm(vmin = -np.max(np.abs(X_pos[i])), 
                               vcenter = 0, 
                               vmax = np.max(np.abs(X_pos[i])) )
    axes[i].imshow(X_pos[i], cmap='bwr', norm=norm)
    axes[i].set_title('postive episode {}'.format(i))

fig, ax = plt.subplots(1,10,figsize=(20,20))
axes = ax.flatten()
for i in range(10):
    norm = colors.TwoSlopeNorm(vmin = -np.max(np.abs(X_pos[i])), 
                               vcenter = 0, 
                               vmax = np.max(np.abs(X_pos[i])) )
    axes[i].imshow(X_ctrl[i], cmap='bwr', norm=norm)
    axes[i].set_title('control episode {}'.format(i))

#### Statistic Moment 1 -- mean

In [ ]:
X_pos_stat = X_pos.mean(axis=0).transpose((1,0))
X_ctrl_stat = X_ctrl.mean(axis=0).transpose((1,0))
X_all_stat = X_all.mean(axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()

im = ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[0])

im = ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[1])

im = ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12))
fig.colorbar(im, ax=ax[2])


### Statistic Moment 2 -- std

In [ ]:
X_pos_stat = X_pos.std(axis=0).transpose((1,0))
X_ctrl_stat = X_ctrl.std(axis=0).transpose((1,0))
X_all_stat = X_all.std(axis=0)[:,:,0].transpose((1,0))

norm = colors.TwoSlopeNorm(vmin = -max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ), 
                               vcenter = 0, 
                               vmax = max( np.max(np.abs(X_all_stat)),np.max(np.abs(X_pos_stat)),np.max(np.abs(X_ctrl_stat)) ) )


fig, axes = plt.subplots(3,1, figsize=(20,20))
ax = axes.flatten()
im=ax[0].imshow(X_pos_stat, cmap='bwr', norm=norm)
ax[0].set_title("Positive Episode")
ax[0].tick_params(axis='y', labelrotation=0)
ax[0].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[0].set_yticklabels(bsi_stream.engineer.input_vars);
ax[0].set_xticks(range(0,96,12))
ax[0].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[0])

im=ax[1].imshow(X_ctrl_stat, cmap='bwr', norm=norm)
ax[1].set_title("Control Group Episode")
ax[1].tick_params(axis='y', labelrotation=0)
ax[1].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[1].set_yticklabels(bsi_stream.engineer.input_vars);
ax[1].set_xticks(range(0,96,12))
ax[1].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[1])

im=ax[2].imshow(X_all_stat, cmap='bwr', norm=norm)
ax[2].set_title("Combined Episode")
ax[2].tick_params(axis='y', labelrotation=0)
ax[2].set_yticks(range(len(bsi_stream.engineer.input_vars)))
ax[2].set_yticklabels(bsi_stream.engineer.input_vars);
ax[2].set_xticks(range(0,96,12))
ax[2].set_xticklabels(range(-96,0,12));
fig.colorbar(im, ax=ax[2])

#### Statistic Moment 3 -- skewness

#### Statistic Moment 4 -- Kurtosis